# Test Code for the Galactic Plane Footprint Metric

Code adapted from an example by Lynne Jones

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import rubin_sim.maf as maf
from rubin_sim.data import get_data_dir
import healpy as hp
from astropy.io import fits

Load the baseline v2.0 as a test case OpSim:

In [2]:
from rubin_sim.data import get_baseline

opsim_fname = get_baseline()
print(opsim_fname)

runName = os.path.split(opsim_fname)[-1].replace('.db', '')
print(runName)
opsim_db = maf.OpsimDatabase(opsim_fname)

/Users/rstreet1/rubin_sim_data/sim_baseline/baseline_v2.0_10yrs.db
baseline_v2.0_10yrs


Taking an example sky location within the Galactic Bulge:

In [3]:
test_ra = (17.0 + 57.0/60.0 + 34.0/3600.0)*15.0
test_dec = (29.0 + 13.0/60.0 + 15.0/3600.0)*-1.0
test_slicer = maf.UserPointsSlicer(test_ra, test_dec)

In [4]:
class galPlaneFootprintMetric(maf.BaseMetric):
    """Metric to evaluate the survey overlap with desired regions in the Galactic Plane
    and Magellanic Clouds, by referencing the pre-computed priority maps provided.

    Parameters
    ----------
    fieldRA : float, RA in degrees of a given pointing
    fieldDec : float, Dec in degrees of a given pointing
    filter : str, filter bandpass used for a given observation
    """

    def __init__(
        self,
        cols=["fieldRA", "fieldDec", "filter", "fiveSigmaDepth"],
        metricName="GalPlaneFootprintMetric",
        **kwargs
    ):
        """Kwargs must contain:
        science_map   string  Name of the priority footprint map to use from
                                the column headers contained in the
                                priority_GalPlane_footprint_map_data tables
        """

        self.ra_col = "fieldRA"
        self.dec_col = "fieldDec"
        self.filterCol = "filter"
        self.m5Col = "fiveSigmaDepth"
        self.filters = ["u", "g", "r", "i", "z", "y"]
        self.science_map = kwargs['science_map']
        self.magCuts = {
            "u": 22.7,
            "g": 24.1,
            "r": 23.7,
            "i": 23.1,
            "z": 22.2,
            "y": 21.4,
        }
        cwd = os.getcwd()
        self.MAP_DIR = get_data_dir()
        self.MAP_FILE_NAME = "priority_GalPlane_footprint_map_data_sum.fits"
        self.load_maps()

        super().__init__(col=cols, metricName=metricName, metricDtype="object")

    def load_maps(self):
        self.NSIDE = 64
        self.NPIX = hp.nside2npix(self.NSIDE)
        file_path = os.path.join(
            self.MAP_DIR,
            "maf",
            self.MAP_FILE_NAME,
            )
        with fits.open(file_path) as hdul:
            self.map_data_table = hdul[1].data

    def run(self, dataSlice, slicePoint):
        """Metric extracts the scientific priority of the given HEALpixel from
        the prepared maps of the desired survey footprint for galactic science.
        The priority map used has been summed over all filters for the
        science case indicated by the kwargs.  This is normalized using the
        summed priority for the map combining the priorities of all science
        cases."""

        metric_data = {}

        pixPriority = self.map_data_table[self.science_map][slicePoint['sid']]

        iObservations = []
        for f in self.filters:
            idx = list(np.where(dataSlice[self.m5Col] >= self.magCuts[f])[0])
            if len(idx) > 0:
                iObservations += idx

        metric_data['nObservations'] = len(iObservations)
        metric_data['priority'] = pixPriority

        return metric_data

In [5]:
mymetric = galPlaneFootprintMetric(science_map='combined_map')

Constraints, if any:

In [6]:
sqlconstraint = None

Construct the metric bundle from the metric, the slicer and the constraints, and add it to a bundle group:

In [7]:
bundle = maf.MetricBundle(mymetric, test_slicer, sqlconstraint, runName=runName)
g = maf.MetricBundleGroup({'test_metric': bundle}, opsim_db, outDir='test', resultsDb=None)

Calculate the metric:

In [8]:
g.runAll()

Querying table None with no constraint for columns ['filter', 'fieldRA', 'rotSkyPos', 'fieldDec', 'fiveSigmaDepth'].
Found 2086980 visits
Running:  ['test_metric']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.


In [9]:
bundle.metricValues

masked_array(data=[{'nObservations': 3293, 'priority': 6.418181e-07}],
             mask=[False],
       fill_value=-666,
            dtype=object)